In [1]:
import pandas as pd

In [5]:
df = pd.read_csv("spam_ham_dataset.csv")
df.head(1)

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0


In [6]:
df = df[ ['text','label_num'] ]
df.head(1)

,text,label_num
0,Subject: enron methanol ; meter # : 988291\r\n...,0


In [14]:
# nltk -> Natural Language Toolkit
!pip install nltk

In [15]:
# Stopword kaldırma -> The,is,at
# Stem -> Köküne indirgemek - Running -> Run
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [29]:
text = df.iloc[0,0]
text

"Subject: enron methanol ; meter # : 988291\r\nthis is a follow up to the note i gave you on monday , 4 / 3 / 00 { preliminary\r\nflow data provided by daren } .\r\nplease override pop ' s daily volume { presently zero } to reflect daily\r\nactivity you can obtain from gas control .\r\nthis change is needed asap for economics purposes ."

In [33]:
# Subject - subject - SUBJECT - Subject: - Subject.
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()

text = df.iloc[0,0] # 1. satırdaki text

def clean_text(text):
    text = text.lower()
    
    text = text.translate( str.maketrans('','',string.punctuation) )
    
    # "subject,enron,methanol" -> ["subject","enron", "methanol"]
    # bir parametre yoksa default boşluk parametresi alınır.
    # merhaba dünya
    words = text.split()
    
    words = [ word for word in words if not word.isdigit() ]
    
    words = [ ps.stem(word) for word in words if word not in stopwords.words('english') ]
    
    return " ".join(words)

clean_text(df.iloc[0,0])

'subject enron methanol meter follow note gave monday preliminari flow data provid daren pleas overrid pop daili volum present zero reflect daili activ obtain ga control chang need asap econom purpos'

In [34]:
df['clean_text'] = df['text'].apply(clean_text)

In [35]:
df.head(1)

,text,label_num,clean_text
0,Subject: enron methanol ; meter # : 988291\r\n...,0,subject enron methanol meter follow note gave ...


In [39]:
X = df['clean_text']
y = df['label_num']

# Vectorize işlemi nasıl olur ? TFIDF Vectorizer nasıl çalışır.
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(df['clean_text'])

X_tfidf

<5171x37932 sparse matrix of type '<class 'numpy.float64'>'
	with 307420 stored elements in Compressed Sparse Row format>

In [41]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.3, random_state=42)

In [43]:
# Naive Bayes
#             spam ham
# ücretsiz -> 80%  10%
# hediye ->   60%   5%

# "Ücretsiz Hediye"
# 0.8 * 0.6

from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train,y_train)

MultinomialNB()

In [45]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)

print(classification_report(y_test,y_pred))

# support => VERİ SAYISI 

# precision -> Kesinlik ->> Söylediğim ne kadar doğru?
# recall -> duyarlılık -> ne kadarını yakaladım?

              precision    recall  f1-score   support

           0       0.89      1.00      0.94      1121
           1       1.00      0.68      0.81       431

    accuracy                           0.91      1552
   macro avg       0.95      0.84      0.88      1552
weighted avg       0.92      0.91      0.91      1552



In [ ]:
# Bu modeli siz geliştiriyor olsaydınız, hangi değişiklikler ile modeli performansını geliştirmek için hangi adımları atardınız. 
